In [145]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from LP import LP
from DeepLP import DeepLP
from DeepLP_Sparse import DeepLP_Sparse
from DeepLP_RBF import DeepLP_RBF
from DeepLP_WeightedRBF import DeepLP_WeightedRBF
from DeepLP_WeightedRBF_t import DeepLP_WeightedRBF_t
import pandas as pd

from utils import *

In [146]:
nodes = np.loadtxt('cora/cora_selected.csv',delimiter=',')
graph = np.loadtxt('cora/cora_graph.csv',delimiter=',')

In [10]:
id_    = nodes[:,0]
labels = nodes[:,1]
data   = nodes[:,2:]

In [11]:
LX, Ly, UX, Uy, Uy_sol = random_unlabel(data,labels)

In [12]:
features = np.vstack((LX,UX))
weights = rbf_kernel(features,G=graph)

/Users/kojin/projects/graph_based_semi_supervised_learning/utils.py:41: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if G == None:


In [40]:
def tnorm(W):
    T = W / np.sum(W, axis=0)
    Tnorm = T / np.sum(T, axis=1)
    return Tnorm

def closed(self,W,Ly):
    n = len(Ly)
    Tnorm = tnorm(W)
    Tuu_norm = Tnorm[n:,n:]
    Tul_norm = Tnorm[n:,:n]
    Uy_lp = inv((np.identity(len(Tuu_norm))-Tuu_norm)) @ Tul_norm @ Ly
    return Uy_lp

In [154]:
T = weights / np.sum(weights, axis=0, keepdims=1).reshape(1,num_nodes)
Tnorm = T / np.sum(T, axis=1, keepdims=1).reshape(num_nodes,1)

# LP

In [55]:
Uy_lp_iter = np.rint(lp.iter_(weights,Ly,Uy,100000))

In [7]:
lp = LP()
Uy_lp = np.rint(lp.closed(weights,Ly))
Uy_lp_iter = np.rint(lp.iter_(weights,Ly,Uy,40))

LinAlgError: Singular matrix

In [47]:
print("[Accuracy] Closed solution:", accuracy(Uy_sol, Uy_lp), " Iter Solution:", accuracy(Uy_sol, Uy_lp_iter))
print("[RMSE] Closed solution", rmse(Uy_sol, Uy_lp), " Iter solution", rmse(Uy_sol, Uy_lp))

NameError: name 'Uy_lp' is not defined

In [ ]:
Uy_lp_iter

In [56]:
print( " Iter Solution:", accuracy(Uy_sol, Uy_lp_iter))
print( " Iter solution", rmse(Uy_sol, Uy_lp_iter))

 Iter Solution: 0.467511238251
 Iter solution 336.910502656


In [60]:
Uy_lp_iter[Uy_lp_iter > 1]

array([   2.,    5.,    2.,    5.,    3.,    3.,    2.,    4.,    2.,
          3.,    9.,    2.,    4.,    2.,    2.,    2.,    8.,    2.,
         12.,    9.,    5.,    4.,    5.,    6.,    2.,    2.,    2.,
          5.,    3.,    2.,    5.,    5.,    2.,    2.,    2.,    3.,
          5.,    4.,    8.,   63.,    5.,   11.,    3.,    2.,    6.,
          9.,    2.,    2.,    2.,    3.,    4.,    5.,    2.,    3.,
          2.,    2.,    3.,    2.,    4.,    9.,   10.,   11.,    6.,
          7.,    3.,   65.,    2.,    2.,   28.,   12.,    4.,    4.,
          4.,   27.,    5.,    3.,    8.,    7.,   23.,    3.,    2.,
         11.,    2.,    5.,    4.,    5.,    2.,   10.,    5.,   18.,
          7.,    2.,    8.,    4.,   12.,    7.,    4.,    2.,   32.,
          4.,    3.,    4.,    2.,    2.,    4.,    3.,    4.,    3.,
          5.,    3.,   12.,    3.,    4.,   11.,   18.,  888.,   64.,
          2.,    3.,    5.,    3.,    2.,    2.,    3.,    4.,    2.,
          3.,    6.,

# DeepLP

In [148]:
true_labels = np.hstack((Ly,Uy_sol))
num_features = features.shape[1]
num_labeled = len(Ly)
num_unlabeled = len(Uy)
num_nodes = num_labeled+num_unlabeled

In [ ]:
# prepare features for NN
LY = np.tile(Ly,(Ly.shape[0],1))
np.fill_diagonal(LY, 0.5)
UY = np.tile(Uy,(Ly.shape[0],1))

masked_ = np.hstack((np.identity(LY.shape[0]),np.zeros((Ly.shape[0],Uy.shape[0]))))
true_labeled = np.array([1] * LY.shape[0] + [0] * Uy.shape[0]).reshape(1,num_nodes)

In [ ]:
unlabeled_test = np.hstack((np.zeros(LY.shape[0]),np.ones((Uy.shape[0])))).reshape(1,num_nodes)
test_data = {
    'X': np.hstack((Ly,Uy)).reshape(1,num_nodes),
    'y': np.tile(true_labels,(Ly.shape[0],1))[1:2],
    'unlabeled': unlabeled_test,
    'labeled': (1 - unlabeled_test).reshape(1,num_nodes),
    'true_labeled': true_labeled,
    'masked':masked_[0:1]
}

In [ ]:
unlabeled_ = np.hstack((np.identity(LY.shape[0]),np.ones((Ly.shape[0],Uy.shape[0]))))
data = {
    'X':np.hstack((LY, UY)),
    'y':np.reshape(true_labels,(1,len(true_labels))),
    'unlabeled':unlabeled_,
    'labeled':1-unlabeled_,
    'true_labeled': true_labeled,
    'masked':masked_
}

In [ ]:
dlp = DeepLP(40, num_nodes, weights, 1)

In [ ]:
pred_dlp = dlp.labelprop(test_data)
Uy_dlp = np.rint(pred_dlp[:,num_labeled:]).reshape(num_unlabeled)

In [ ]:
print("Accuracy:", accuracy(Uy_sol,Uy_dlp))
print("RMSE:", rmse(Uy_sol,Uy_dlp))
print("Objective:", objective(Ly,Uy_dlp,dlp.get_val(dlp.W)))

# DeepLP_RBF

In [ ]:
dlp_rbf = DeepLP_RBF(40, num_nodes, features, graph, 1, 0.01)

In [ ]:
pred_dlp_rbf = dlp_rbf.labelprop(test_data)
Uy_dlp_rbf = np.rint(pred_dlp_rbf[:,num_labeled:]).reshape(num_unlabeled)

In [ ]:
print("Accuracy:", accuracy(Uy_sol,Uy_dlp_rbf))
print("RMSE:", rmse(Uy_sol,Uy_dlp_rbf))
print("Objective:", objective(Ly,Uy_dlp_rbf,dlp_rbf.get_val(dlp_rbf.W)))

In [19]:
Uy_dlp_rbf

array([  0.,   1.,   0., ...,   1.,   2.,  12.], dtype=float32)

In [20]:
graph

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [91]:
dlp_rbf.get_val(dlp_rbf.W),dlp.get_val(dlp.W)

(array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32),
 array([[ 0.        ,  0.13533528,  0.00247875, ...,  0.04978707,
          0.04978707,  0.00673795],
        [ 0.13533528,  0.        ,  0.        , ...,  0.36787945,
          0.00673795,  0.        ],
        [ 0.00247875,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.00673795],
        ..., 
        [ 0.04978707,  0.36787945,  0.        , ...,  0.        ,
          0.01831564,  0.00247875],
        [ 0.04978707,  0.00673795,  0.        , ...,  0.01831564,
          0.        ,  0.01831564],
        [ 0.00673795,  0.        ,  0.00673795, ...,  0.00247875,
          0.01831564,  0.        ]], dtype=float32))